In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GenAI para segmentação de clientes com BigQuery

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

Segmentação de clientes é o processo de agrupar clientes base em características similares para ajudar no engajamento de uma marca ou de um produto com sua audiência. Isso pode ser feito usando uma variedade de fatores, como dados demográficos e comportamentais. Mecanismos de *machine learning* podem ser usados para automatizar o processo de segmentação de clientes, tornando-o mais eficiente e eficaz.

### BigQuery e o BigQueryML

O [BigQuery](https://cloud.google.com/bigquery/). é o serviço de base de dados gerenciado, serverless e escalável disponível na Google Cloud. O BigQuery permite gerenciar e analisar dados com recursos integrados, como *machine learning*, análise geoespacial e *business intelligence*. Para maiores detalhes, visite a [documentação oficial](https://cloud.google.com/bigquery/docs) do BigQuery.

O [BigQueryML](https://cloud.google.com/bigquery/docs/bqml-introduction?hl=pt-br) é o componente do BigQuery para criar e executar modelos de *machine learning* usando consultas com código GoogleSQL, permitindo profissionais de SQL criem modelos usando habilidades e ferramentas de SQL já existentes e sem a necessidade de movimentação de dados para fora do banco de dados. Para maiores detalhes, visite a [documentação oficial](https://cloud.google.com/bigquery/docs/bqml-introduction?hl=pt-br) do BigQueryML.


### Vertex AI PaLM API
A Vertex AI PaLM API, [lançada em 10 de maio de 2023](https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes#may_10_2023), é desenvolvida com [PaLM 2]( https://ai.google/discover/palm2).

### Usando a API Vertex AI PaLM

Você pode interagir com a API Vertex AI PaLM usando os seguintes métodos:

* Use a UI da [Generative AI Studio](https://cloud.google.com/generative-ai-studio) para testes rápidos e geração de comandos.
* Faça chamadas REST no Cloud Shell.
* Use o Python SDK em um notebook Jupyter

Este notebook se concentra no uso do Python SDK para chamar a Vertex AI PaLM API. Para obter mais informações sobre como usar o Generative AI Studio sem escrever código, você pode explorar [Introdução às instruções da interface do usuário](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/getting-started/getting_started_ui.md)

Para obter mais informações, confira a [documentação sobre suporte de IA generativa para Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Neste tutorial, você irá utilizar os serviços de Google Cloud para:

* Explorar dados do dataset público `bigquery-public-data.thelook_ecommerce.order_items`
* Treinar um modelo de segmentação utilizando o algoritmo `kmeans` no BigQueryML
* Realizar inferências no modelo treinado
* Utilizar as API do GenAI Studio para a geração de campanhas utilizando as informações do modelo do BigQueryML

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Generative AI Studio
* BigQuery
* BigQueryML

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
[preços do BigQuery](https://cloud.google.com/bigquery/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

### Segurança de dados
**P: O Google usa dados de clientes para melhorar seus modelos de base?**
R: Não, o Google não usa dados de clientes para melhorar os modelos de fundação. Os dados do cliente são usados apenas para gerar uma resposta do modelo.

**P: Os funcionários do Google veem os dados que envio ao modelo?**
R: Não, os funcionários do Google não têm acesso aos dados do cliente e todos os dados são criptografados em trânsito, em uso e em repouso.

**P: O Google armazena algum dos dados do cliente que são enviados para o modelo?**
R: Não, o Google não armazena dados de clientes. No entanto, o Google pode armazenar em cache temporariamente os dados do cliente durante a solicitação, como pipeline de ajuste de prompt e uso em batch.

**P: O Google registra dados?**
R: Não, o Google não loga os dados dos clientes. Os logs do lado do sistema ajudam o Google a garantir a integridade e a disponibilidade do sistema.

### IA Responsável
LLMs podem traduzir linguagem, resumir texto, gerar escrita criativa, gerar código, chatbots e assistentes virtuais, além de complementar mecanismos de pesquisa e sistemas de recomendação. Ao mesmo tempo, como uma tecnologia em estágio inicial, seus recursos e usos em evolução podem criar aplicações incorretas, uso indevido e consequências não intencionais ou imprevistas. LLMs podem gerar resultados inesperados, incluindo texto ofensivo, insensível ou incorreto.

Além disso, a incrível versatilidade dos LLMs também é o que torna difícil prever exatamente que tipos de saídas não intencionais ou imprevistas eles podem produzir. Dados esses riscos e complexidades, a API PaLM foi projetada com os [AI Principles do Google](https://ai.google/principles/) em mente. No entanto, é importante que os desenvolvedores entendam e testem seus modelos para implantá-los com segurança e responsabilidade. Para ajudar os desenvolvedores, o Generative AI Studio possui filtragem de conteúdo integrada e a API PaLM possui pontuação de atributo de segurança para ajudar os clientes a testar os filtros de segurança do Google e definir limites de confiança adequados para seu caso de uso e negócios. Consulte a seção [Filtros e atributos de segurança](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#safety_filters_and_attributes) para saber mais.

Quando a API PaLM é integrada ao caso de uso e contexto exclusivos de um cliente, considerações adicionais de IA Responsável e [limitações PaLM](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai #palm_limitations) precisam ser considerados. Incentivamos os clientes a usar *fairness*, interpretabilidade, privacidade e segurança [práticas recomendadas](https://ai.google/responsabilidades/responsible-ai-practices/).

## Primeiros Passos

### Instalando os SDK da Vertex AI e da Cloud Translate API

In [1]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 88.5 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-pubsublite 1.8.2 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.3.1 which is incompatible.
ray 2.5.0 requires grpcio<=1.51.3,>=1.42.0; python_version >= "3.10" and sys_platform != "darwin", but you have grpcio 1.56.0 which is incompatible.


**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [2]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [3]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [4]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [18]:
import sys
import pandas as pd
from typing import Union
from google.cloud import bigquery
from vertexai import language_models
from langchain.llms.base import LLM
from langchain.embeddings.base import Embeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders.dataframe import DataFrameLoader
from vertexai.preview.language_models import TextGenerationModel

from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.embeddings.base import Embeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains.summarize import load_summarize_chain

## Geração de texto com `text-bison@001`

O modelo de geração de texto da API PaLM que você usará neste notebook é `text-bison@001`. Já deixaremos seu objeto instanciado neste Jupyter notebook para uso futuro

#### Carregando o modelo `text-bison`

In [9]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

#### Criando a função *wrapper* para utilizar os modelos em Português

Até o momento desde treinamento, as API do Generative AI Studio suportam somente interações no idioma inglês. Para fazermos interações utilizando o idioma português, vamos utilizar a [Cloud Translation API](https://cloud.google.com/translate) para traduzir as nossas solicitações do português para o inglês e as respostas da API de inglês para português.

In [10]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

### Preparando as funções necessárias 

In [11]:
#@title Define LLM classes and instantiate
import time

from google.cloud import aiplatform
from vertexai import language_models

from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from langchain.document_loaders.dataframe import DataFrameLoader
from langchain.indexes import VectorstoreIndexCreator


def rate_limit(max_per_minute):
  period = 60 / max_per_minute
  while True:
    before = time.time()
    yield
    after = time.time()
    elapsed = after - before
    sleep_time = max(0, period - elapsed)
    if sleep_time > 0:
      print(f'Sleeping {sleep_time:.1f} seconds')
      time.sleep(sleep_time)


class VertexEmbeddings(Embeddings):

  def __init__(self, model, *, requests_per_minute=15):
    self.model = model
    self.requests_per_minute = requests_per_minute

  def embed_documents(self, texts):
    limiter = rate_limit(self.requests_per_minute)
    results = []
    docs = list(texts)

    while docs:
      # Working in batches of 2 because the API apparently won't let
      # us send more than 2 documents per request to get embeddings.
      head, docs = docs[:2], docs[2:]
      chunk = self.model.get_embeddings(head)
      results.extend(chunk)
      next(limiter)

    return [r.values for r in results]

  def embed_query(self, text):
    single_result = self.embed_documents([text])
    return single_result[0]


class VertexLLM(LLM):

  model: language_models.TextGenerationModel
  predict_kwargs: dict

  def __init__(self, model, **predict_kwargs):
    super().__init__(model=model, predict_kwargs=predict_kwargs)

  @property
  def _llm_type(self):
    return 'vertex'

  def _call(self, prompt, stop=None):
    result = self.model.predict(prompt, **self.predict_kwargs)
    return str(result)

  @property
  def _identifying_params(self):
    return {}

# NOTE: Use staging to get 100qps max throughput for embedding indexing
# The embedding content is the same as production so you can use staging
# for indexing and production for querying if desired.
#language_models.TextEmbeddingModel._LLM_ENDPOINT_NAME = (
#  'projects/678515165750/locations/us-central1/endpoints/8156038716377268224')

REQUESTS_PER_MINUTE = 6000

model = language_models.TextGenerationModel.from_pretrained('google/text-bison@001')
llm = VertexLLM(
  model,
  max_output_tokens=256,
  temperature=0.1,
  top_p=0.8,
  top_k=40
)
embedding = VertexEmbeddings(language_models.TextEmbeddingModel.from_pretrained('google/textembedding-gecko@001'), requests_per_minute=REQUESTS_PER_MINUTE)

llm

VertexLLM(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, model=<vertexai.language_models._language_models.TextGenerationModel object at 0x7fcc19852ef0>, predict_kwargs={'max_output_tokens': 256, 'temperature': 0.1, 'top_p': 0.8, 'top_k': 40})

In [12]:
project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id

client = bigquery.Client(project=project_id)
dataset_id = f"{project_id}.bigquery_llm"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, timeout=30)
print("Dataset criado com sucesso {}.{}".format(project_id, dataset.dataset_id))

Dataset criado com sucesso lucianomartins-demos-345000.bigquery_llm


In [13]:
query = """
SELECT
   refresh_date AS Day,
   term AS Top_Term,
   rank,
FROM `bigquery-public-data.google_trends.top_terms`
WHERE
   rank <= 3
   # AND refresh_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK)
GROUP BY Day, Top_Term, rank
ORDER BY Day DESC, rank
"""

client = bigquery.Client(project=project_id)
df = client.query(query).to_dataframe()
df

,Day,Top_Term,rank
0,2023-07-06,Threads,1
1,2023-07-06,Hong Kong singer Coco Lee,2
2,2023-07-06,Rudy Farias,3
3,2023-07-05,Cocaine found in White House,1
4,2023-07-05,Philadelphia shooting,2
...,...,...,...
91,2023-06-06,Google Maps,2
92,2023-06-06,Maps,3
93,2023-06-05,Miami Heat,1
94,2023-06-05,The Idol,2


In [15]:
# Limit the size of the table due to context window
googletrends_table = df.head(5).to_markdown()

prompt = f"""
  Use the following table in markdown format to answer the question.

  {googletrends_table}

  Generate a sentence that describe the top-3 trends from 2023-07-05
"""

# Compare results by adding this to the last line of prompt: "For example, top search trends for Apr 24, 2023 is Warriors"
response = model.predict(prompt)
response

The top 3 trends on 2023-07-05 are:
1. Cocaine found in White House
2. Philadelphia shooting
3. Threads

In [16]:
import pandas as pd

data = []
for index, rows in df.iterrows():
  text = """On {0}, top {1} search term is {2}.
  """.format(
      rows['Day'],
      rows['rank'],
      rows['Top_Term'])
  data.append(text)

# Put it in a dataframe so we can easily index it
nl_googletrends_df = pd.DataFrame(data, columns=['text'])
nl_googletrends_df

,text
0,"On 2023-07-06, top 1 search term is Threads.\n"
1,"On 2023-07-06, top 2 search term is Hong Kong ..."
2,"On 2023-07-06, top 3 search term is Rudy Faria..."
3,"On 2023-07-05, top 1 search term is Cocaine fo..."
4,"On 2023-07-05, top 2 search term is Philadelph..."
...,...
91,"On 2023-06-06, top 2 search term is Google Map..."
92,"On 2023-06-06, top 3 search term is Maps.\n"
93,"On 2023-06-05, top 1 search term is Miami Heat..."
94,"On 2023-06-05, top 2 search term is The Idol.\n"


In [17]:
persistent_path = './dados-locais'
df_loader = DataFrameLoader(nl_googletrends_df, page_content_column="text")

googletrends_index = VectorstoreIndexCreator(
    embedding=embedding, 
    vectorstore_kwargs={
        'persist_directory': f'{persistent_path}/googletrends'
    }).from_loaders([df_loader])

In [72]:
def ask_question(question, max_results=5, threshold=0.5):
    # Based on the question, sarch for relevant articles
    similar_docs = googletrends_index.vectorstore.similarity_search_with_score(question, llm=llm, k=max_results)
    filtered_docs = list(filter(lambda doc: doc[1] <= threshold, similar_docs))
    context = "\n".join([doc.page_content for doc, score in filtered_docs])
    prompt = f"""
    
    {context}
    
    Answer based on the context data only.
    Explain why it's a top search term particularly on that day, what happened that day?
    Don't make up information. Don't say information you are not sure about.

    Question: {question}
    Answer:
    """
    return model.predict(prompt)

In [73]:
ask_question('What is top 1 search term on 2023-07-04')

The top 1 search term on 2023-07-04 is Fourth of July. This is because the Fourth of July is a national holiday in the United States, and people often search for information about the holiday, such as what events are happening in their area, how to celebrate, and what the history of the holiday is.

In [74]:
ask_question('What was second top search term on 2023-06-13?')

Treat Williams is an American actor. He was born on 1948-09-11. He is best known for his roles in the films "Hair", "The Prince of Tides", and "The Postman". He has also starred in the television series "Everwood", "The Practice", and "Blue Bloods". On 2023-06-13, Treat Williams was 74 years old.

In [76]:
ask_question('When was the search term "LSU Baseball" on the top-3 search terms?')

The search term "LSU Baseball" was on the top-3 search terms on 2023-06-26. This is because LSU Baseball was playing in the College World Series on that day, and the game was being televised nationally.